In [19]:
import numpy as np
from matplotlib import pyplot as plt
import scipy.stats as stats
import pandas as pd
import numpy as nm
import math
import statistics as st



from pymle.models import OrnsteinUhlenbeck 
from pymle.sim.Simulator1D import Simulator1D 
from pymle.TransitionDensity import ExactDensity 
from pymle.fit.AnalyticalMLE import AnalyticalMLE 
import numpy as np 
 

In [39]:
N = 1000000
T = 1
t = np.linspace(0, T, N + 1)
Delta = np.sqrt(T / N)
x0 = 10


In [21]:
def Ornstein_Uhlenbeck_EM(T, N, t, Delta, x0, w):
    kappa = 3
    alpha = 1
    sigma = 2
    x = np.zeros(N + 1)
    x[0] = x0
    for i in range(1, N + 1):
        x[i] = x[i-1] + kappa * (alpha - x[i-1]) * T / N + sigma * w[i-1]
    return x

In [22]:
def get_param(x, delta, N):
    el1 = np.zeros(N-1) 
    el5 = np.zeros(N-1) 
    x1 = np.zeros(N-1)
    for i in range(0, N - 1):
        x1[i] = x[i + 1]
    for i in range(0, N-1):
        el1[i] = x1[i] * x[i]
    sum1 = np.sum(el1) + x1[N-2] * x[N-2]
    el2 = np.sum(x)
    el3 = np.sum(x) - x[N]
    el4 = np.sum(x ** 2)   
    beta1 = (sum1 - el2 * el3 * N) / (el4 - el3 ** 2 * N) 
    # print(beta1)
    
    for i in range(0, N - 1): 
        el5[i] = x1[i]- beta1 * x[i]
    sum2 = np.sum(el5)
    # print('sum2 =', sum2)
    beta2 = sum2/ (1 - beta1) / N 
    # el6 = 0 
    # for i in range(1, N): 
    #     el6+=(X[i] - beta1*X[i-1] - beta2*(1-beta1))**2 
    # beta3=N**(-1)*el6 

    kappa = (-1 / delta) * np.log(beta1) 
    alpha = beta2
    # alpha = beta2 
    # sigma = np.sqrt(2*kappa*beta3*(1-beta1**2)**(-1)) 

    return kappa, alpha

In [23]:
w = np.random.normal(0, np.sqrt(T / N), N + 1)
x = Ornstein_Uhlenbeck_EM(T, N, t, Delta, x0, w)
get_param(x, 1 / N, N)

(-0.300914981399258, 33.22425370657477)

In [24]:
def get_params(x, delta, N):
    sum1 = np.sum(x)
    
    sum2 = np.sum(x ** 2)
#     print(sum2, sum1)

    sum31 = np.zeros(N)
    x1 = np.zeros(N)
    for i in range(0, N - 1):
        x1[i] = x[i + 1]
    
    for i in range(0, N-1):
        sum31[i] = x1[i] * x[i]
    sum3 = np.sum(sum31)
#     print(sum3)
    theta2 = - np.log(sum3 / sum2) / delta
    
    sum41 = np.zeros(N)
    for i in range(0, N):
        sum41[i] = (x1[i] - x[i] * np.exp(-delta * theta2)) ** 2
    sum4 = np.sum(sum41)
    theta3 = 2 * theta2 * sum4 / (N * (1 - np.exp(-2 * delta * theta2)))
    

    return(theta2 * N, np.sqrt(theta3 * N))



In [25]:
w = np.random.normal(0, np.sqrt(T / N), N + 1)
x = Ornstein_Uhlenbeck_EM(T, N, t, Delta, x0, w)
theta2_test = 0
theta3_test = 0
# for i in range(0,100):
a, b = get_params(x, 1, N)
#     theta2_test += a
#     theta3_test += b
# print(theta2_test / 100)
# print(theta3_test / 100)
print(a,b)

2.7296359048517296 2.054076138069959


In [64]:
x0 = 2
N = 1000
t = np.linspace(0, T, 1001)
def CKLS_model(T, N, t, Delta, x0, w):
    theta1 = 1
    theta2 = 2
    theta3 = 0.5
    theta4 = 0.3
    x = np.zeros(N+1)
    x[0] = x0
    delta = 1 / 10000
    for i in range(1, N+1):
        x[i] = x[i-1] + (theta1 + theta2 * x[i-1]) * T / N + theta3 * x[i-1] ** theta4 * w[i-1]
    return x

def get_CKLS(x, N):
    a = np.zeros(N)
    for i in range(1, N + 1):
        a[i-1] = x[i] - x[i-1]
    b = np.mean(a) * N
    sig = np.sqrt(st.variance(a)) * N
    return a

def get_confidence_intervals(x, alpha):
    a = stats.t.interval(alpha, df=len(x)-1, loc=np.mean(x), scale=stats.sem(x))
    return a



In [50]:
w = np.random.normal(0, np.sqrt(T / N), N + 1)
x = CKLS_model(T, N, t, Delta, x0, w)
a = get_CKLS(x, N)
get_confidence_intervals_CKLS(x, 0.9)

(6.935851593202957, 7.338464731312094)

In [61]:
def params():
    model = OrnsteinUhlenbeck()   
    kappa = 3  # rate of mean reversion 
    mu = 1  # long term level of process 
    sigma = 2  # volatility 
    theta = 3

    model.params = np.array([kappa, mu, sigma]) 

    S0 = 10  # initial value of process 
    T = 1000  # number of days of the sample 
    freq = 1  # observations per day 
    dt = 1 / 1000
    seed = None  # random seed: set to None to get new results each time 

    simulator = Simulator1D(S0=S0, M=T * freq, dt=dt, model=model).set_seed(seed=seed) 
    sample = simulator.sim_path() 

    param_bounds = [(0, 6), (0, 5), (0, 5), (0, 7)] 
    guess = np.array([0.5, 0.1, 0.4, 0.5]) 
    exact_est = AnalyticalMLE(sample, param_bounds, dt, density=ExactDensity(model)).estimate_params(guess) 

    return(exact_est.params)

In [73]:
kappa = []
alpha = []
sigma = []
for i in range(0, 100):
    a = params()
    kappa.append(a[0])
    alpha.append(a[1])
    sigma.append(a[2])

kappa_int = get_confidence_intervals(kappa, 0.8)
alpha_int = get_confidence_intervals(alpha, 0.7)
sigma_int = get_confidence_intervals(sigma, 0.95)



Initial Params: [0.5 0.1 0.4]
Initial Likelihood: -10431.832123002601
`gtol` termination condition is satisfied.
Number of iterations: 37, function evaluations: 124, CG iterations: 75, optimality: 5.72e-07, constraint violation: 0.00e+00, execution time: 0.078 s.
Final Params: [2.75121594e+00 1.90124115e-05 2.13361871e+00]
Final Likelihood: 1278.494619154501
Initial Params: [0.5 0.1 0.4]
Initial Likelihood: -9622.152580409109
`xtol` termination condition is satisfied.
Number of iterations: 71, function evaluations: 256, CG iterations: 157, optimality: 8.63e-06, constraint violation: 0.00e+00, execution time: 0.15 s.
Final Params: [2.88008554 0.00636713 2.06257821]
Final Likelihood: 1312.4216952112317
Initial Params: [0.5 0.1 0.4]
Initial Likelihood: -8936.906469479993
`gtol` termination condition is satisfied.
Number of iterations: 48, function evaluations: 168, CG iterations: 107, optimality: 7.88e-07, constraint violation: 0.00e+00, execution time: 0.11 s.
Final Params: [2.88726831 0

Initial Params: [0.5 0.1 0.4]
Initial Likelihood: -8014.992082338496
`xtol` termination condition is satisfied.
Number of iterations: 56, function evaluations: 200, CG iterations: 128, optimality: 1.38e-05, constraint violation: 0.00e+00, execution time: 0.13 s.
Final Params: [2.59863429 0.32168848 1.91814684]
Final Likelihood: 1384.8783275491699
Initial Params: [0.5 0.1 0.4]
Initial Likelihood: -9776.147841823404
`gtol` termination condition is satisfied.
Number of iterations: 50, function evaluations: 172, CG iterations: 110, optimality: 1.35e-07, constraint violation: 0.00e+00, execution time: 0.12 s.
Final Params: [3.54529388e+00 7.17563306e-07 2.06971631e+00]
Final Likelihood: 1309.2991441971535
Initial Params: [0.5 0.1 0.4]
Initial Likelihood: -9048.360547815111
`xtol` termination condition is satisfied.
Number of iterations: 59, function evaluations: 240, CG iterations: 136, optimality: 5.74e-06, constraint violation: 0.00e+00, execution time: 0.13 s.
Final Params: [3.08672287 1

`xtol` termination condition is satisfied.
Number of iterations: 58, function evaluations: 240, CG iterations: 130, optimality: 4.72e-06, constraint violation: 0.00e+00, execution time: 0.14 s.
Final Params: [4.30723676 1.31975486 1.97265587]
Final Likelihood: 1357.7103815402743
Initial Params: [0.5 0.1 0.4]
Initial Likelihood: -8966.898842184139
`xtol` termination condition is satisfied.
Number of iterations: 58, function evaluations: 220, CG iterations: 130, optimality: 5.43e-06, constraint violation: 0.00e+00, execution time: 0.12 s.
Final Params: [4.41441825 2.98782064 2.00991839]
Final Likelihood: 1339.050573382975
Initial Params: [0.5 0.1 0.4]
Initial Likelihood: -9106.314823606053
`xtol` termination condition is satisfied.
Number of iterations: 55, function evaluations: 188, CG iterations: 118, optimality: 1.04e-05, constraint violation: 0.00e+00, execution time: 0.11 s.
Final Params: [2.70229557e+00 3.78545650e-06 2.02317832e+00]
Final Likelihood: 1331.6199438637645
Initial Par

Initial Params: [0.5 0.1 0.4]
Initial Likelihood: -8735.698668232875
`gtol` termination condition is satisfied.
Number of iterations: 48, function evaluations: 168, CG iterations: 106, optimality: 8.13e-07, constraint violation: 0.00e+00, execution time: 0.11 s.
Final Params: [3.30852494 0.22478137 1.98158226]
Final Likelihood: 1352.6968043442384
Initial Params: [0.5 0.1 0.4]
Initial Likelihood: -8972.156819834734
`xtol` termination condition is satisfied.
Number of iterations: 56, function evaluations: 168, CG iterations: 130, optimality: 9.27e-06, constraint violation: 0.00e+00, execution time:  0.1 s.
Final Params: [3.63835813 3.02514657 2.02072061]
Final Likelihood: 1333.3029988009612
Initial Params: [0.5 0.1 0.4]
Initial Likelihood: -9022.218281267522
`gtol` termination condition is satisfied.
Number of iterations: 67, function evaluations: 240, CG iterations: 144, optimality: 2.11e-07, constraint violation: 0.00e+00, execution time: 0.16 s.
Final Params: [5.43640987 1.83069699 1.

Initial Params: [0.5 0.1 0.4]
Initial Likelihood: -9811.03360946096
`xtol` termination condition is satisfied.
Number of iterations: 53, function evaluations: 160, CG iterations: 111, optimality: 5.58e-06, constraint violation: 0.00e+00, execution time: 0.11 s.
Final Params: [4.7061309  2.72972802 2.07474286]
Final Likelihood: 1307.4530945839833
Initial Params: [0.5 0.1 0.4]
Initial Likelihood: -9172.249222812907
`xtol` termination condition is satisfied.
Number of iterations: 60, function evaluations: 224, CG iterations: 138, optimality: 5.42e-06, constraint violation: 0.00e+00, execution time: 0.13 s.
Final Params: [4.40426051 2.15295141 2.01834084]
Final Likelihood: 1334.8638081979875
Initial Params: [0.5 0.1 0.4]
Initial Likelihood: -8227.99615518849
`gtol` termination condition is satisfied.
Number of iterations: 53, function evaluations: 184, CG iterations: 113, optimality: 2.06e-07, constraint violation: 0.00e+00, execution time: 0.11 s.
Final Params: [3.24604204 1.84693239 1.94

In [74]:
kappa_int

(3.2188571288476515, 3.4242346924026954)

In [75]:
alpha_int

(1.05988884455269, 1.2402880682685078)

In [76]:
sigma_int

(1.9868011201951208, 2.0055865943613536)